In [1]:
import pandas as pd
import numpy as np

In [2]:
import sklearn
from sklearn import linear_model, svm
from sklearn.metrics import classification_report

In [3]:
textscores_df = pd.read_csv("textscores_1671191407926.csv")
textscores_df.rename(columns={"label": "true_label", "pred": "textlabel", "pred_score": "textscore"}, inplace=True)

linkscores_df = pd.read_csv("linkscore.csv")
engagementscores_df = pd.read_csv("engagementscores.csv")
userscores_df = pd.read_csv("userscores.csv")

data = pd.merge(textscores_df, linkscores_df, on="tweetId").merge(engagementscores_df, on="tweetId").merge(userscores_df, on="tweetId")

data_train = data.loc[data.split == "train"].reset_index(drop=True)
data_val = data.loc[data.split == "val"].reset_index(drop=True)

In [4]:
x_cols = ["textscore", "linkscore", "engagementscore", "userscore"]
y_col = "true_label"

In [5]:
data

,tweetId,split,true_label,textlabel,textscore,linkscore,engagementlabel,engagementscore,userlabel,userscore
0,253611215190896642,train,1,1,0.959302,2.0,-1,-0.262399,1,0.359550
1,443584873048309760,val,1,1,0.999068,2.0,1,1.000000,1,0.358064
2,618437306769215488,train,1,1,0.991276,2.5,-1,-0.470649,1,0.354376
3,642098739272773632,val,-1,-1,-0.733052,2.0,1,1.000000,1,0.363188
4,712708069369782272,val,1,1,0.841321,2.5,1,1.000000,1,0.337059
...,...,...,...,...,...,...,...,...,...,...
10775,1603168527191506944,train,-1,-1,-0.991609,4.0,-1,-0.121793,-1,-0.291264
10776,1603168547152318471,train,-1,-1,-0.998769,4.0,-1,-0.121793,-1,-0.313816
10777,1603168554710204417,train,-1,-1,-0.999677,3.0,-1,-0.121793,1,0.334152
10778,1603168595370024960,train,-1,-1,-0.994042,3.0,-1,-0.121793,1,0.362238


In [6]:
model = linear_model.LogisticRegression()
model.fit(data_train[x_cols], data_train[y_col])
preds = model.predict(data_val[x_cols])
print(classification_report(data_val[y_col], preds, target_names=["Not Misleading", "Misleading"]))

                precision    recall  f1-score   support

Not Misleading       0.97      0.98      0.97      1425
    Misleading       0.96      0.95      0.95       731

      accuracy                           0.97      2156
     macro avg       0.96      0.96      0.96      2156
  weighted avg       0.97      0.97      0.97      2156



In [7]:
print("Model accuracy:", sklearn.metrics.accuracy_score(data_val[y_col], preds))
print("Model precision:", sklearn.metrics.precision_score(data_val[y_col], preds))
print("Model recall:", sklearn.metrics.recall_score(data_val[y_col], preds))

Model accuracy: 0.9666048237476809
Model precision: 0.9557399723374828
Model recall: 0.945280437756498


In [8]:
np.exp(model.coef_)

array([[37.94179169,  0.41136234,  9.0050974 ,  1.30965132]])